In [3]:
import pandas as pd
import os
os.chdir(r"/Users/apple/Documents/Credit_Card_Churn_Model_Local/credit-card-churn")
#Initialising Dataset
data = pd.read_csv("/Users/apple/Documents/Credit_Card_Churn_Model_Local/credit-card-churn/cleaned_credit_card_churn_numeric_feature_selection.csv") 

In [9]:
y = data['Attrition_Flag']
X = data.drop(['Attrition_Flag'], axis=1)

In [15]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
smote = SMOTE(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

/Users/apple/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
